# Mancala: Player vs AI
In this project we have implemented a playable version of mancala using python with **slight rule tweaks**. We then proceeded to go try and test a few different types of AI to use as an opposing "computer" player for the humans to play against. These strategies were tested in this order:
1. Random moves.
2. Minimax
3. Minimax with alpha beta pruning

Our goal is to produce an AI that plays signficantly better than the random AI while running in real time without the user or player having to wait for long periods of time while the AI computes.
## Rule changes to our mancala
Our game of Mancala plays much like the normal version of mancala. We have varying sized boards that don't impact gameplay, but our main tweak to the original Mancala is:
1. If your last stone for your turn lands in your Mancala, you don't get to make another move. In other words, the turn always alternates every time a player makes a move without exception.
## How to play
In the cell below we define a series of variables with comments explaining what they do. You can simply toggle on your preferred variables and then run the code. If a player is playing, you will have to send your moves through the CLI by typing a valid number seen in the printed Mancala board.

In [ ]:
PLAY_GAME = False # temp variable, whether or not to play the basic mancala we implemented in hw7. Useful to avoid the "input" prompt coming up everytime we run the code
MOVE_COUNT = 10 # number of moves (both players combined) before the game ends automatically

# Libraries used
Our libraries used are imported below. Most importantly, we are using the AIMA library game abstract class such that we will also have access to their minimax and alpha beta pruning algorithms.

In [ ]:
import random
from random import seed
from random import randint
random.seed(109)

In [ ]:
from games4e import *
from utils4e import *

# Basic interactive Mancala game vs random player
Below is a simple implementation of Mancala with no superclass. It defines a variable-sized 2-player game using the rule exceptions we defined above. It uses a random move algorithm as the AI player.

In [ ]:
class Mancala:
    def __init__(self, pits_per_player=3, stones_per_pit = 2):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        
        # write your code here
        if(self.current_player == 1):
            if (pit > 0 and pit < (self.pits_per_player + 1)) and (self.board[pit-1] > 0): 
                return True
            else:
                return False
        elif(self.current_player == 2):
            if (pit > 0 and pit < (self.pits_per_player + 1)) and (self.board[self.pits_per_player + pit] > 0): 
                return True
            else:
                return False
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        
        # write your code here
        value = randint(1, self.pits_per_player)
        while(self.board[value + self.pits_per_player] == 0):
            value = randint(1, self.pits_per_player)
        return value
    
    def play(self, pit):
            """
            This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
            1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
            2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
            3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

            Finally, the function then switches the current player, allowing the other player to take their turn.
            """
            print("Player", self.current_player, "chooses pit:", pit)
            # 1. Check valid move
            if not self.valid_move(pit):
                print("INVALID MOVE")
                return
            
            # DISTRIBUTE STONES
            if self.current_player == 1:
                start = self.p1_pits_index[0] + pit - 1
            else:
                start = self.p2_pits_index[0] + pit - 1

            stones = self.board[start]
            self.board[start] = 0

            while stones > 0:
                start = (start + 1) % len(self.board)  # Move to the next pit, wrapping around
                if self.current_player == 1 and start == self.p2_mancala_index:
                    continue  # Skip opponent's Mancala
                if self.current_player == 2 and start == self.p1_mancala_index:
                    continue  # Skip opponent's Mancala
                
                self.board[start] += 1
                stones -= 1

            # Capture stones if the last stone lands in an empty pit on the player's side
            if self.current_player == 1 and self.p1_pits_index[0] <= start <= self.p1_pits_index[1]:
                if self.board[start] == 1:  # Empty pit
                    opposite_pit = self.p2_pits_index[1] - (start - self.p1_pits_index[0])
                    self.board[self.p1_mancala_index] += self.board[start] + self.board[opposite_pit]
                    self.board[start] = 0
                    self.board[opposite_pit] = 0
            elif self.current_player == 2 and self.p2_pits_index[0] <= start <= self.p2_pits_index[1]:
                if self.board[start] == 1:  # Empty pit
                    opposite_pit = self.p1_pits_index[1] - (start - self.p2_pits_index[0])
                    self.board[self.p2_mancala_index] += self.board[start] + self.board[opposite_pit]
                    self.board[start] = 0
                    self.board[opposite_pit] = 0

            self.moves.append((self.current_player, pit))
            
            # 3. Switch player
            self.current_player = 2 if self.current_player == 1 else 1
            
            # 2. Winning evaluation, check for winner
            if self.winning_eval():
                print("GAME OVER")
                return

            return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        p1_empty = self.board[self.p1_pits_index[0]:self.p1_pits_index[1] + 1] == [0] * self.pits_per_player
        p2_empty = self.board[self.p2_pits_index[0]:self.p2_pits_index[1] + 1] == [0] * self.pits_per_player
        return p1_empty or p2_empty

In [ ]:
# Mancala part 2
import sys
game = Mancala(6, 4)
movecount = MOVE_COUNT # num of moves to allow in total
game.display_board()

# Loop to play game via user input if PLAU_GAME is True
if PLAY_GAME:
    for i in range(0, movecount):
        pit = None
        if game.winning_eval():
            print("Game ended, no more valid moves for Player", game.current_player)
            break

        if game.current_player == 1:
            try:
                pit = int(input("Player 1, choose a pit: ").strip())
                print("Input received:", pit)
            except:
                print("Invalid input. Try again")
            if not pit or not game.valid_move(pit):
                print("Invalid move \"" + str(pit) + "\". Try again")
                continue
        else:
            pit = game.random_move_generator()
        if pit:
            game.play(pit)
            game.display_board()
            print()
        else:
            print("No valid moves for Player", game.current_player)
            break

    # Game ended, print information
    print("\nList of valid moves:")
    for move in game.moves:
        player, pit = move
        print(f"Player {player} selected pit {pit}")


# Subclassing AIMA Game for Mancala